In [49]:
# Cargar librerías
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
import mlflow
import mlflow.sklearn
from flask import Flask, request, jsonify


In [50]:
# Leer datos
df = pd.read_csv("data/diabetes.csv")

In [51]:
# identifica las columnas que tiene valores 0
(df==0).sum()

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

In [52]:
# Reemplazar ceros en columnas específicas por NaN
cols_to_clean = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin']
# No se consideran algunas, por ejemplo Pregnancies (por qué si es posible que se tengan cero embarazos)

# Reemplazar ceros por NaN para poder tratarlos como datos faltantes
for col in cols_to_clean:
    df[col] = df[col].replace(0, np.nan)

# Reemplazar NaN con la moda (valor más frecuente) de cada columna
for col in cols_to_clean:
    # buscar el método más común para determinar la moda de una columna 
    moda = df[col].mode()
    df[col] = df[col].fillna(moda)


In [75]:
# identifica las columnas que tiene valores 0
(df==0).sum()

Pregnancies                 111
Glucose                       0
BloodPressure                 0
SkinThickness                 0
Insulin                       0
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

In [76]:
# reemplazar los NaN en el dataset
df = df.interpolate(method='linear')
# confirmar que no existen NaN
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [77]:
# Separar datos
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'Data Split done.')

Data Split done.


In [78]:
# agregar el traking uri de mlflow
mlflow.set_tracking_uri("http://localhost:9090") # cambiar en función de su servidor

# agregar set_experiment con nombre: ClasificadorDemoDiabetes
# Create a new MLflow Experiment
mlflow.set_experiment("ClasificadorDemoDiabetes")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1747661783027, experiment_id='1', last_update_time=1747661783027, lifecycle_stage='active', name='ClasificadorDemoDiabetes', tags={}>

In [104]:
# tomar datos de logreg para las iteraciones de mlflow
# log = pd.read_csv("data/logreg_variaciones_educativas.csv")
import csv
with open('data/logreg_variaciones_educativas.csv', newline='') as csvfile:
    log = csv.reader(csvfile, delimiter=',')
    header = next(log) # Skips the header if it exists
    print(header)
    for row in log:
        print(row[2])

['run_id', 'logreg_C', 'logreg_max_iter', 'solver', 'penalty']
400
300
500
200
300
500
300
500
400
200


In [113]:
# Entrenamiento y registro con MLflow
# tomar datos de logreg para las iteraciones de mlflow
# log = pd.read_csv("data/logreg_variaciones_educativas.csv")

#C = 1.0
# max_iter = 10 #1000
# usar los parámetros del archivo logreg_variaciones_educativas.csv
# para los parámtros C, max_iter, solver y penalty de LogisticRegresion
# se debe genear un run por cada fila del archivo, usando sus parámetros
# Analizar que usar

# header csv: ['run_id', 'logreg_C', 'logreg_max_iter', 'solver', 'penalty']
import csv
with open('data/logreg_variaciones_educativas.csv', newline='') as csvfile:
    log = csv.reader(csvfile, delimiter=',')
    next(log) # Skips the header
    for row in log:
    # agregar la línea del start_run
        with mlflow.start_run(run_name=row[0]):
            # El pipeline usar StandarScaler para que todos los valore numéricos estén en la misma escala
            # no cambiar dicha línea
            # LogisticRegresion deben variar sus parámetros en cada iteración.
            pipeline = Pipeline([
                    ("scaler", StandardScaler()),
                    ("clf", LogisticRegression(
                        C=float(row[1]),
                        max_iter=int(row[2]),
                        solver=row[3],
                        penalty= row[4] #"l2"
                    ))
                ])
            
            # Entrenar y evaluar
            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)
        
            acc = accuracy_score(y_test, y_pred)
            prec = precision_score(y_test, y_pred)
        
            # recuerde que log_param y log_metric debe ir guardando los valores
            # dependiendo del run correspondiente de cada fila
            mlflow.log_param("logreg_C", C)
            mlflow.log_param("logreg_max_iter", max_iter)
            mlflow.log_param("solver", "liblinear")
            mlflow.log_param("penalty", "l2")
            
            mlflow.log_metric("accuracy", acc)
            mlflow.log_metric("precision", prec)
            
            # Guardar el pipeline completo
            mlflow.sklearn.log_model(pipeline, "modelo_pipeline")
            
            print(" Modelo registrado en MLflow")
            print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f}")


2025/05/19 12:28:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\WNMH84\HerramientasIA\clase6\env\Lib\site-packages\mlflow\data\digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead."
2025/05/19 12:28:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\WNMH84\HerramientasIA\clase6\env\Lib\site-packages\mlflow\types\utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling 

 Modelo registrado en MLflow
Accuracy: 0.7273 | Precision: 0.6140


2025/05/19 12:28:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\WNMH84\HerramientasIA\clase6\env\Lib\site-packages\mlflow\types\utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/05/19 12:28:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\WNMH84\HerramientasIA\clase6\

 Modelo registrado en MLflow
Accuracy: 0.7597 | Precision: 0.6731


2025/05/19 12:28:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\WNMH84\HerramientasIA\clase6\env\Lib\site-packages\mlflow\types\utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/05/19 12:28:44 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\WNMH84\HerramientasIA\clase6\

...

 Modelo registrado en MLflow
Accuracy: 0.7597 | Precision: 0.6731


2025/05/19 12:30:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\WNMH84\HerramientasIA\clase6\env\Lib\site-packages\mlflow\types\utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/05/19 12:30:14 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\WNMH84\HerramientasIA\clase6\

 Modelo registrado en MLflow
Accuracy: 0.7597 | Precision: 0.6731


2025/05/19 12:30:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\WNMH84\HerramientasIA\clase6\env\Lib\site-packages\mlflow\types\utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/05/19 12:30:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\WNMH84\HerramientasIA\clase6\

 Modelo registrado en MLflow
Accuracy: 0.7597 | Precision: 0.6731
